In [1]:
import pandas as pd

In [13]:
train = pd.read_csv('./train.csv', index_col=0)
test = pd.read_csv('./test.csv', index_col=0)

train.shape, test.shape

((1460, 80), (1459, 79))

In [11]:
train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
# Data cleaning

In [8]:
# Number of missing values in each column of training data
missing_val_count_by_column = (train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64


In [15]:
missing_val_count_by_column = (test.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

MSZoning           4
LotFrontage      227
Alley           1352
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        16
MasVnrArea        15
BsmtQual          44
BsmtCond          45
BsmtExposure      44
BsmtFinType1      42
BsmtFinSF1         1
BsmtFinType2      42
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu      730
GarageType        76
GarageYrBlt       78
GarageFinish      78
GarageCars         1
GarageArea         1
GarageQual        78
GarageCond        78
PoolQC          1456
Fence           1169
MiscFeature     1408
SaleType           1
dtype: int64


In [ ]:
# I decided to delete columns which have more than 50% nulls

In [20]:
train.drop(missing_val_count_by_column[missing_val_count_by_column >= 0.5*train.shape[0]].index, 
           inplace = True, axis=1)
test.drop(missing_val_count_by_column[missing_val_count_by_column >= 0.5*train.shape[0]].index, 
           inplace = True, axis=1)

train.shape

(1460, 75)

In [23]:
# Depending on variable type, I'm imputting most frequent value for categorical features and median for numerical features

In [24]:
# median or mean?

In [33]:
target_f = ['SalePrice']
cat_f = train.iloc[:, :-1].select_dtypes(exclude='number').columns
num_f = train.iloc[:, :-1].select_dtypes(include='number').columns
len(cat_f) +len(num_f) + 1

75

In [38]:
from sklearn.impute import SimpleImputer

impute_median = SimpleImputer(strategy='median')
train_num_imputed = pd.DataFrame(impute_median.fit_transform(train.loc[:, num_f]), columns=num_f, index=train.index)
test_num_imputed = pd.DataFrame(impute_median.transform(test.loc[:, num_f]), columns=num_f, index=test.index)

impute_freq = SimpleImputer(strategy='most_frequent')
train_cat_imputed = pd.DataFrame(impute_freq.fit_transform(train.loc[:, cat_f]), columns=cat_f, index=train.index)
test_cat_imputed = pd.DataFrame(impute_freq.transform(test.loc[:, cat_f]), columns=cat_f, index=test.index)

In [43]:
train_num_imputed.shape

(1460, 36)

In [48]:
train_imputed = pd.concat([train_num_imputed, train_cat_imputed], axis = 1)
test_imputed = pd.concat([test_num_imputed, test_cat_imputed], axis = 1)

train_imputed.shape

(1460, 74)

In [ ]:
# convert categorical to binary with LabelEncoder & OneHotEncoder

In [ ]:
# ile na ile podzielić

In [ ]:
# choosing variables for analysis

In [52]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score

#L2 liblinear  1722
logistic = LogisticRegression(C=1, penalty="l2", random_state=13, solver='lbfgs').fit(train_imputed, train.SalePrice)
model = SelectFromModel(cross_val_score(logistic), prefit=True)
X_new = model.transform(train_df)
selected_features = pd.DataFrame(model.inverse_transform(X_new), 
                                 index=train_df.index,
                                 columns=train_df.columns)

# Dropped columns have values of all 0s, keep other columns 
L2_selected_columns = selected_features.columns[selected_features.var() != 0]
len(L2_selected_columns)

ValueError: could not convert string to float: 'Normal'